In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import time
import math
#from torchinfo import summary
import os
import time

In [3]:
def laplace(y, x):
    grad = gradient(y, x)
    return divergence(grad, x)


def divergence(y, x):
    div = 0.
    for i in range(y.shape[-1]):
        div += torch.autograd.grad(y[..., i], x, torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
    return div


def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, x, grad_outputs=grad_outputs, create_graph=True, allow_unused=True)[0]
    return grad

In [4]:
class FactorizedGlorotNormal(nn.Module):
    def __init__(self, mean=1.0, stddev=0.1):
        super(FactorizedGlorotNormal, self).__init__()
        self.mean = mean
        self.stddev = torch.tensor(stddev)

    def forward(self, shape):
        w =  torch.nn.init.xavier_normal_(torch.empty(shape))#.normal_(mean=0, std=1)
        s = torch.exp(self.mean + torch.empty((shape[-1],)).normal_(mean=0, std=self.stddev))
        v = w / s
        return  s, v


class FactorizedDense(nn.Module):
    def __init__(self, in_features, out_features):
        super(FactorizedDense, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.kernel = FactorizedGlorotNormal()
        self.bias = nn.Parameter(torch.zeros(out_features))

        self.s = nn.Parameter(self.kernel((self.in_features, self.out_features))[0])
        self.v = nn.Parameter(self.kernel((self.in_features, self.out_features))[1])

    def forward(self, x):
        #s, v = self.kernel((self.in_features, self.out_features))
        kernel = self.s * self.v
        y = torch.matmul(x, kernel) + self.bias
        return y


class FourierFeatureEmbedding(nn.Module):
    def __init__(self, input_dim, embedding_dim, sigma=3.0):
        super(FourierFeatureEmbedding, self).__init__()
        self.B = torch.randn((embedding_dim // 2, input_dim)) * sigma #* 2 * torch.pi

    def forward(self, x):
        x_proj = 2 * torch.pi * x @ self.B.t()
        x_proj = torch.deg2rad(x_proj)
        #print(x.shape, self.B.shape, x_proj.shape)
        return torch.cat([torch.cos(x_proj), torch.sin(x_proj)], dim=1)


class Stanh(nn.Module):
    def __init__(self, feat_num):
        super(Stanh, self).__init__()
        self.a = nn.Parameter(torch.zeros(feat_num))

    def forward(self, x):
        output = torch.tanh(x)  + self.a * x * torch.tanh(x)
        return output

def dist(x1, y1, x2, y2):
    return torch.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def linseg(x, y, x1, y1, x2, y2):
    L = dist(x1, y1, x2, y2)
    xc = (x1 + x2) / 2.
    yc = (y1 + y2) / 2.
    f = (1 / L) * ((x - x1) * (y2 - y1) - (y - y1) * (x2 - x1))
    t = (1 / L) * ((L / 2.)**2 - dist(x, y, xc, yc)**2)
    varphi = torch.sqrt(t**2 + f**4)
    phi = torch.sqrt(f**2 + (1 / 4.) * (varphi - t)**2)
    return phi

def phi(x, y, segments):
    m = 1.
    R = 0.
    for i in range(segments.size(0)):
        x1, y1, x2, y2 = segments[i]
        phi_val = linseg(x, y, x1, y1, x2, y2)
        R = R + 1. / phi_val**m
    R = 1. / R**(1. / m)
    return R

segments = torch.tensor([[-0.5, -0.5, 0.5, -0.5], [0.5, -0.5, 0.5, 0.5], [0.5, 0.5, -0.5, 0.5], [-0.5, 0.5, -0.5, -0.5]])

class Pinn(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_layers, output_dim, sigma=1.0, lr=1e-3, fourier=True):
        super(Pinn, self).__init__()
        self.fourier = fourier

        if self.fourier:
            self.fourier_features = FourierFeatureEmbedding(input_dim, embedding_dim, sigma)
            input_dim = embedding_dim  

        self.layers = nn.ModuleList()
        self.acts = []

        current_dim = input_dim
        for hidden_dim in hidden_layers:
            self.layers.append(nn.Linear(current_dim, hidden_dim))
            self.acts.append(Stanh(hidden_dim))
            current_dim = hidden_dim

        self.layers.append(nn.Linear(current_dim, output_dim))
        self.layers.apply(self.init_weights)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr)

    def init_weights(self, m):
      if isinstance(m, nn.Linear):
          torch.nn.init.xavier_uniform_(m.weight)
          torch.nn.init.normal_(m.bias)

    def forward(self, x, y):
        xy = torch.cat([x,y], dim=1)
      
        dist = ((xy[:, 0:1] - 0.5) * (xy[:, 0:1] + 0.5) * (xy[:, 1:2] - 0.5) * (xy[:, 1:2] + 0.5))

        if self.fourier:
            xy = self.fourier_features(xy)

        for i in range(len(self.layers) - 1):
            xy = self.acts[i](self.layers[i](xy))
        xy = self.layers[-1](xy)

        w, phix, phiy, phix_x_plus_phiy_y, phix_y_plus_phiy_x = xy.split(1, dim=1)

        w = w*dist**2
        return torch.cat([w, phix, phiy, phix_x_plus_phiy_y, phix_y_plus_phiy_x], dim=1)

In [5]:
class Pde():
    def __init__(self, E, nu=0.3, h=0.1, k=5/6):

        G = E/(2*(1+nu))

        self.h = h
        self.k = k

        self.D = (E*h**3)/(12*(1-nu**2))
        self.kGh = k*G*h
        self.Dc = (2*self.kGh)/(self.D*(1-nu))
        self.Ds = k*G*h
        self.Db = (E*h**3)/(12*(1-nu**2))

        print(E, G, self.D, self.Dc)
        self.criterion = nn.MSELoss()


    def lossWeight(self, losses):
        with torch.no_grad():
          total = 0.0
          weights = []
          for i in range(len(losses)):
            total += i

          for i in range(len(losses)):
            weight = i/total
            weights.append(1-weight)
        return weights

    def getDerivs(self, w, phix, phiy):

        phix_x = gradient(phix, x)
        phiy_y = gradient(phiy, y)

        phix_y = gradient(phix, y)
        phiy_x = gradient(phiy, x)

        phix_x_plus_phiy_y = phix_x + phiy_y
        phix_y_minus_phiy_x = phix_y - phiy_x

        derivs = torch.cat([phix_x_plus_phiy_y, phix_y_minus_phiy_x], dim=1)
        return derivs

    def getStresses(self, bstrains, sstrains):
        bstresses = torch.matmul(self.Df, bstrains.T)
        sstresses = torch.matmul(self.Dc, sstrains.T)
        return bstresses.T, sstresses.T

    def getLoss(self, model, x, y):

        output = model(x,y)
        w, phix, phiy, phix_x_plus_phiy_y, phix_y_minus_phiy_x = output.split(1, dim=1)

        derivs_pred = torch.cat([phix_x_plus_phiy_y, phix_y_minus_phiy_x], dim=1)
        target_derivs = self.getDerivs(w, phix, phiy)

        derivloss = self.criterion(derivs_pred, target_derivs)

        w_lap = laplace(w, x) + laplace(w, y)

        q = 1.0

        eq1 = (laplace(phix_x_plus_phiy_y, x) + laplace(phix_x_plus_phiy_y, y)) + q/self.D
        eq2 = w_lap + phix_x_plus_phiy_y + q/(self.kGh*self.k)
        eq3 = (laplace(phix_y_minus_phiy_x, x) + laplace(phix_y_minus_phiy_x,y)) - (12*self.k*2/self.h**2)*(phix_y_minus_phiy_x)


        res1 = self.criterion(eq1, torch.zeros_like(eq1))
        res2 = self.criterion(eq2, torch.zeros_like(eq2))
        res3 = self.criterion(eq3, torch.zeros_like(eq3))
        resloss = res1 + res2 + res3

        return resloss, derivloss

In [22]:
def plot_loss(model, comp):
    with torch.no_grad():
        x_vals = torch.linspace(-0.5, 0.5, steps=100)
        y_vals = torch.linspace(-0.5, 0.5, steps=100)
        X, Y = torch.meshgrid(x_vals, y_vals, indexing='xy')
        Z = torch.zeros_like(X)

        input_data = torch.stack([X.flatten(), Y.flatten(), Z.flatten()], dim=1)
        solution_w = model(X.flatten().unsqueeze(1), Y.flatten().unsqueeze(1))[:, 0:1]
        solution_w = solution_w.reshape(X.shape)

        fem = comp
        max_w = torch.abs(solution_w).max().item()
        rel_diff = (abs(max_w - fem) / fem) * 100
        print(f'Max w: {max_w}, FEM: {fem}, Relative Difference at center: {rel_diff:.2f}%')

        plt.scatter(X, Y, c=solution_w, cmap='jet')
        plt.xlabel("Length (m)")
        plt.ylabel("Width (m)")
        plt.colorbar().set_label('w (m)')
        plt.title("Mindlin Plate")
        plt.gca().set_aspect('equal', adjustable='box')
        plt.show()

def plot_loss2(model, loss):
    x = range(len(loss))
    plot_generic(x, loss, "Iterations", "Residual Loss", "Loss", log=False)


def internalPoints(length, scale, steps):
    z = torch.linspace(-length, length, steps) * scale
    lower_bound = torch.min(z)
    upper_bound = torch.max(z)

    dz = (upper_bound - lower_bound) / (steps-1)
    new_start = dz/2

    new_coords = torch.linspace(lower_bound + new_start, upper_bound - new_start, steps-1)
    return new_coords, dz


def plot_generic(x, y, xlabel, ylabel, title, log=False):
    with torch.no_grad():
        plt.scatter(x, y, c='red')
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        #plt.xlim(left=-0.5, right=0.5)
        #plt.ylim(bottom=-0.5, top=0.5)
        plt.title(title)
        if log:
          plt.yscale('log')
        plt.show()


def getData(num_points, grad=False):
  # Number of points
  num_points = 80
  # Generate random values sampled from a uniform distribution in the range [0, 1]
  x = torch.rand(num_points, 1, requires_grad=grad)
  x = x * (0.49 - (-0.49)) + (-0.49)
  y = torch.rand(num_points, 1, requires_grad=grad)
  y = y * (0.49 - (-0.49)) + (-0.49)
  return x, y

In [ ]:
pde = Pde(E=10920, nu=0.3, h=0.1, k=5/6)

iterations = 100001
report_step = 1000

total_time = []
total_error = []
iterations_list = []
exp_accuracy = []


best_metric = 20
for j in range(1):
  residual_loss = []
  rel_error = []
  best_avg_rel_error = 100
  timer = 0.0
  rel_diff = 100.0

  patience = 150
  epochs_without_improvement = 0
  dV =0.0

  input_dim = 2  
  embedding_dim = 32  
  hidden_layers = [32, 32, 32, 32]  
  output_dim = 5 
  model = Pinn(input_dim, embedding_dim, hidden_layers, output_dim, sigma=10.0, lr=1.0e-3)

  iteration_time_start = time.time()
  dV = 0.0
  fem = 10.0

  for i in range(iterations):

        model.optimizer.zero_grad()
       
        x, y = getData(num_points=40, grad=True)

        resloss, derivloss = pde.getLoss(model, x, y)

        loss = resloss + derivloss
        loss.backward()
        model.optimizer.step()
        #model.scheduler.step(loss)

        with torch.no_grad():
            residual_loss.append(resloss.item())
            #print(residual_loss)
          
            if i % report_step ==0:
                print(f'rel_diff {rel_diff}')
                print(f'res loss {resloss.detach().numpy()}')
                plot_loss(model=model, comp=fem)
                plot_loss2(model=model, loss=residual_loss)

            # Checkpoint
            # if rel_diff > 10:
            #   if int(rel_diff) < int(best_metric):
            #       best_metric = rel_diff
            #       # Save the model
            #       best_avg_rel_error = best_metric
            #       torch.save(model.state_dict(), 'best_model.pth')
            #       print(f'Saving model with best rel_diff: {best_metric}')
            # else:
            #   if rel_diff < best_metric:
            #       best_metric = rel_diff
            #       # Save the model
            #       best_avg_rel_error = best_metric
            #       torch.save(model.state_dict(), 'best_model.pth')
            #       print(f'Saving model with best rel_diff: {best_metric}')

  with torch.no_grad():
      plot_loss(model=model, comp=fem)
      print()
      plot_loss2(model, residual_loss)
      print()
      print(f'Experiment {j}, time: {total_time[-1]}, best_accuracy: {total_error[-1]}')